In [1]:
train_path = 'fv2_train.parquet'
BATCH_SIZE = 8192   
NUM_CROSS_LAYERS = 3
LR = 0.001
EPOCHS = 3
model_path = '2_DCN_MLP.pth'
data_folder = 'C:/Users/Николай/PycharmProjects/VKRecSys/custom_data/'

In [2]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

In [3]:
torch.manual_seed(42)  
torch.cuda.manual_seed_all(42)  
np.random.seed(42)  

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
train = pd.read_parquet(f'{data_folder}{train_path}', engine='pyarrow')
train['target'] = train['target'].replace({-1:0, 0:1, 1:2})

In [5]:
class MyDataset(Dataset):
    def __init__(self, interactions, device):
        self.device = device
        self.users = torch.tensor(interactions['user_id'].values, dtype=torch.long, device=self.device)
        self.items = torch.tensor(interactions['item_id'].values, dtype=torch.long, device=self.device)
        self.ages = torch.tensor(interactions['age'].values, dtype=torch.long, device=self.device)
        self.item_durations = torch.tensor(interactions['item_duration'].values, dtype=torch.long, device=self.device)
        
        if 'target' in interactions.columns:
            self.targets = torch.tensor(interactions['target'].values, dtype=torch.float32, device=self.device)
        else:
            self.targets = None  

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        if self.targets is not None:
            return self.users[idx], self.items[idx], self.ages[idx], self.item_durations[idx], self.targets[idx]
        else:
            return self.users[idx], self.items[idx], self.ages[idx], self.item_durations[idx]

In [6]:
train_ds = MyDataset(train, device)
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE)

In [7]:
class DCN(nn.Module):
    def __init__(self, input_dim, num_cross_layers):
        super(DCN, self).__init__()
        self.input_dim = input_dim
        self.num_cross_layers = num_cross_layers
        
        self.cross_weights = nn.ParameterList(
            [nn.Parameter(torch.randn(input_dim, 1)) for _ in range(num_cross_layers)]
        )
        self.cross_biases = nn.ParameterList(
            [nn.Parameter(torch.randn(input_dim)) for _ in range(num_cross_layers)]
        )
        
    def forward(self, x):
        x0 = x
        for i in range(self.num_cross_layers):
            x = x0 * (x @ self.cross_weights[i]) + self.cross_biases[i] + x
        return x

class DCNWithMLP(nn.Module):
    def __init__(self, input_dim, num_cross_layers=3, hidden_dim=128, output_dim=3):
        super(DCNWithMLP, self).__init__()
        
        self.batch_norm = nn.BatchNorm1d(input_dim)
        
        self.dcn = DCN(input_dim, num_cross_layers)
        
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.batch_norm(x)
        
        x = self.dcn(x)

        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))  
        x = self.fc3(x)
        return self.softmax(x)

input_dim = len(train.columns) - 1
num_cross_layers = NUM_CROSS_LAYERS

model = DCNWithMLP(input_dim, num_cross_layers).to(device)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

In [9]:
for epoch in range(EPOCHS):
    running_loss = 0.0
    with tqdm(train_dl, desc=f"Epoch {epoch+1}/{EPOCHS}", unit="batch") as t:
        for users, items, ages, item_durations, targets in t:
            inputs = torch.cat((users.unsqueeze(1), items.unsqueeze(1), ages.unsqueeze(1), item_durations.unsqueeze(1)), dim=1).float()
            targets = targets
            
            optimizer.zero_grad()
            
            outputs = model(inputs)
            
            loss = criterion(outputs, targets.long())
            
            loss.backward()
            
            optimizer.step()
            
            running_loss += loss.item()
            
            t.set_postfix(mean_loss=f"{running_loss / (t.n + 1):.6f}")
    
    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {running_loss/len(train_dl):.4f}")

Epoch 1/3: 100%|██████████| 17782/17782 [42:11<00:00,  7.02batch/s, mean_loss=0.598567] 


Epoch [1/3], Loss: 0.5986


Epoch 2/3: 100%|██████████| 17782/17782 [43:16<00:00,  6.85batch/s, mean_loss=0.598438] 


Epoch [2/3], Loss: 0.5984


Epoch 3/3: 100%|██████████| 17782/17782 [1:01:38<00:00,  4.81batch/s, mean_loss=0.598438]

Epoch [3/3], Loss: 0.5984


In [10]:
torch.save(model.state_dict(), model_path)
print(f"Модель сохранена в {model_path}")

Модель сохранена в 2_DCN_MLP.pth
